<a href="https://colab.research.google.com/github/jeromecamilleri/reseauElectrique/blob/main/SimuElecFinale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
import random
from IPython.display import clear_output
import matplotlib.colors as mcolors
import matplotlib.cm as cm

def build_network(N, prod_ratio=0.1, cons_ratio=0.3):
    prod_count = int(prod_ratio * N)
    cons_count = int(cons_ratio * N)
    all_nodes = list(range(N))

    producers = np.random.choice(all_nodes, prod_count, replace=False)
    remaining = list(set(all_nodes) - set(producers))
    consumers = np.random.choice(remaining, cons_count, replace=False)

    A = np.zeros((N, N))
    for i in range(N):
        targets = set()
        while len(targets) < min(3, N-1):
            j = random.randint(0, N - 1)
            if j != i:
                targets.add(j)
        for j in targets:
            A[i, j] = A[j, i] = -1
    for i in range(N):
        A[i, i] = -np.sum(A[i])

    return A, producers, consumers

def solve_network(A, producers, consumers, I_mean, I_std):
    N = len(A)
    I = np.zeros(N)
    I_cons = np.clip(np.random.normal(I_mean, I_std, len(consumers)), 1, None)
    for i, c in enumerate(consumers):
        I[c] = I_cons[i]
    I_total = I.sum()
    for p in producers:
        I[p] = -I_total / len(producers)

    prod_idx = producers
    other_idx = [i for i in range(N) if i not in prod_idx]

    U_p = np.full(len(prod_idx), 230)
    A_oo = A[np.ix_(other_idx, other_idx)]
    A_op = A[np.ix_(other_idx, prod_idx)]
    I_o = I[other_idx]
    b = I_o - A_op @ U_p
    U_o = np.linalg.solve(A_oo, b)

    U = np.zeros(N)
    U[prod_idx] = U_p
    U[other_idx] = U_o

    return U, I

def plot_network(A, U, producers, consumers, I):
    N = len(A)
    edges = [(i, j) for i in range(N) for j in range(i+1, N) if A[i, j] != 0]

    # Créer un graphe orienté avec le sens du courant
    G = nx.DiGraph()
    G.add_nodes_from(range(N))
    for i, j in edges:
        if U[i] > U[j]:
            G.add_edge(i, j)
        else:
            G.add_edge(j, i)

    pos = nx.spring_layout(G, seed=42)

    node_colors = ['green' if n in producers else 'red' if n in consumers else 'lightgray' for n in range(N)]
    currents = [abs(U[i] - U[j]) for i, j in edges]

    min_c, max_c = min(currents), max(currents)
    widths = np.interp(currents, [min_c, max_c], [0.5, 7]) if max_c != min_c else [3]*len(currents)

    norm = mcolors.Normalize(vmin=min_c, vmax=max_c)
    cmap = plt.colormaps['coolwarm']  # Mise à jour selon Matplotlib 3.7+

    colors = [cmap(norm(c)) for c in currents]

    fig, ax = plt.subplots(figsize=(12, 12))
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=100, ax=ax)
    nx.draw_networkx_edges(G, pos, edgelist=G.edges(), width=widths, edge_color=colors, ax=ax, arrowsize=15)
    nx.draw_networkx_labels(G, pos, font_size=7, ax=ax)
    ax.axis('off')
    ax.set_title("Réseau électrique - producteurs (vert), consommateurs (rouge)")

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, ax=ax, shrink=0.7, label='Intensité du courant (approx.)')
    plt.show()


# Variables globales
network_state = {"A": None, "producers": None, "consumers": None, "N": 100}

def on_nodes_change(N):
    A, producers, consumers = build_network(N)
    U, I = solve_network(A, producers, consumers, I_mean=50, I_std=10)
    network_state.update({"A": A, "producers": producers, "consumers": consumers, "N": N})
    clear_output(wait=True)
    plot_network(A, U, producers, consumers, I)

def on_consumption_change(I_mean=50, I_std=10):
    A = network_state["A"]
    producers = network_state["producers"]
    consumers = network_state["consumers"]
    if A is None:
        return
    U, I = solve_network(A, producers, consumers, I_mean, I_std)
    clear_output(wait=True)
    plot_network(A, U, producers, consumers, I)

# Interact sliders
interact(on_nodes_change, N=IntSlider(min=10, max=200, step=10, value=100, description="Nœuds"))
interact(on_consumption_change,
         I_mean=FloatSlider(min=10, max=200, step=10, value=50, description="Conso moy"),
         I_std=FloatSlider(min=0, max=50, step=5, value=10, description="Écart-type"))


interactive(children=(IntSlider(value=100, description='Nœuds', max=200, min=10, step=10), Output()), _dom_cla…

interactive(children=(FloatSlider(value=50.0, description='Conso moy', max=200.0, min=10.0, step=10.0), FloatS…

<function __main__.on_consumption_change(I_mean=50, I_std=10)>